In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
print(tf.__version__)

2.3.1


In [3]:
imdb, info = tfds.load("imdb_reviews", with_info = True, as_supervised=True)

Shuffling and writing examples to /Users/jd/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePTSEMD/imdb_reviews-train.tfrecord


Shuffling and writing examples to /Users/jd/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePTSEMD/imdb_reviews-test.tfrecord


Shuffling and writing examples to /Users/jd/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePTSEMD/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /Users/jd/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

In [8]:
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []

for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())
    
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [10]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = "post"
oov_tok = "<oov>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()
model2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 120, 16)           160000    
__________________________

In [23]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

num_epochs = 10
model.fit(padded,
         training_labels_final,
         epochs=num_epochs,
         validation_data=(testing_padded,testing_labels_final))

Epoch 1/10
782/782 [==============================] - 1s 2ms/step - loss: 5.1939e-04 - accuracy: 0.9999 - val_loss: 1.6108 - val_accuracy: 0.8046
Epoch 2/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0023 - accuracy: 0.9992 - val_loss: 1.6368 - val_accuracy: 0.8066
Epoch 3/10
782/782 [==============================] - 1s 2ms/step - loss: 4.3567e-04 - accuracy: 0.9999 - val_loss: 1.7098 - val_accuracy: 0.8018
Epoch 4/10
782/782 [==============================] - 1s 2ms/step - loss: 2.3816e-05 - accuracy: 1.0000 - val_loss: 1.6564 - val_accuracy: 0.8067
Epoch 5/10
782/782 [==============================] - 1s 2ms/step - loss: 1.1456e-06 - accuracy: 1.0000 - val_loss: 1.6569 - val_accuracy: 0.8070
Epoch 6/10
782/782 [==============================] - 1s 2ms/step - loss: 7.1055e-07 - accuracy: 1.0000 - val_loss: 1.6579 - val_accuracy: 0.8069
Epoch 7/10
782/782 [==============================] - 1s 2ms/step - loss: 5.7518e-07 - accuracy: 1.0000 - val_loss: 1.6592 - val

In [25]:
model2.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

num_epochs = 10
model2.fit(padded,
         training_labels_final,
         epochs=num_epochs,
         validation_data=(testing_padded,testing_labels_final))

Epoch 1/10
782/782 [==============================] - 1s 2ms/step - loss: 0.5597 - accuracy: 0.7319 - val_loss: 0.3887 - val_accuracy: 0.8387
Epoch 2/10
782/782 [==============================] - 1s 2ms/step - loss: 0.3361 - accuracy: 0.8583 - val_loss: 0.3329 - val_accuracy: 0.8565
Epoch 3/10
782/782 [==============================] - 1s 2ms/step - loss: 0.2743 - accuracy: 0.8887 - val_loss: 0.3332 - val_accuracy: 0.8563
Epoch 4/10
782/782 [==============================] - 1s 2ms/step - loss: 0.2381 - accuracy: 0.9064 - val_loss: 0.3430 - val_accuracy: 0.8533
Epoch 5/10
782/782 [==============================] - 1s 2ms/step - loss: 0.2124 - accuracy: 0.9210 - val_loss: 0.3645 - val_accuracy: 0.8463
Epoch 6/10
782/782 [==============================] - 1s 2ms/step - loss: 0.1907 - accuracy: 0.9304 - val_loss: 0.3847 - val_accuracy: 0.8446
Epoch 7/10
782/782 [==============================] - 1s 2ms/step - loss: 0.1744 - accuracy: 0.9352 - val_loss: 0.4104 - val_accuracy: 0.8397
Epoch 

In [26]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [31]:
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

In [34]:
import io
out_v =io.open('vecs.tsv','w',encoding='utf-8')
out_m = io.open('meta.tsv','w', encoding='utf-8')
for word_num in range(1,vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [35]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 68, 105, 12, 7, 491, 1216]]
